# Visualization of Network Growth in Assembly Processes

This notebook demonstrates how coordination networks assemble over time,
visualizing the evolution of graph structure and emergent properties.

In [ ]:
import sys
sys.path.insert(0, '../..')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

from assembly_net.data.synthetic_assembly_simulator import (
    CoordinationNetworkSimulator,
    SimulationParameters,
    generate_dataset,
)
from assembly_net.utils.visualization import (
    plot_assembly_trajectory,
    plot_network_growth,
    plot_betti_evolution,
    animate_assembly,
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Generate a Single Assembly Trajectory

We'll simulate the assembly of a metal-ligand coordination network.

In [ ]:
# Define simulation parameters
params = SimulationParameters(
    num_metal_ions=25,
    num_ligands=50,
    metal_valency=6,
    ligand_valency=2,
    ph=7.0,
    ionic_strength=0.1,
    base_formation_rate=1.0,
    base_dissociation_rate=0.01,
    total_time=100.0,
    snapshot_interval=1.0,
    seed=42,
)

# Run simulation
simulator = CoordinationNetworkSimulator(params)
trajectory = simulator.run()

print(f"Generated trajectory with {len(trajectory.states)} states")
print(f"Final network: {trajectory.final_state.graph.num_nodes} nodes, {len(trajectory.final_state.graph.edges)} edges")

## 2. Visualize Assembly Snapshots

See how the network grows over time.

In [ ]:
fig = plot_assembly_trajectory(trajectory, num_snapshots=6)
plt.show()

## 3. Network Properties Over Time

Track how structural properties evolve during assembly.

In [ ]:
fig = plot_network_growth(trajectory)
plt.show()

## 4. Topological Evolution

Track Betti numbers (connected components and cycles) over time.

In [ ]:
fig = plot_betti_evolution(trajectory)
plt.show()

## 5. Compare Different Assembly Conditions

See how pH and concentration affect network formation.

In [ ]:
conditions = [
    {'ph': 5.0, 'label': 'Low pH (5.0)'},
    {'ph': 7.0, 'label': 'Neutral pH (7.0)'},
    {'ph': 9.0, 'label': 'High pH (9.0)'},
]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, cond in zip(axes, conditions):
    params = SimulationParameters(
        num_metal_ions=20,
        num_ligands=40,
        ph=cond['ph'],
        total_time=100.0,
        seed=42,
    )
    
    sim = CoordinationNetworkSimulator(params)
    traj = sim.run()
    
    # Plot edges over time
    times = [s.time for s in traj.states]
    edges = [len(s.graph.edges) for s in traj.states]
    
    ax.plot(times, edges, linewidth=2)
    ax.set_xlabel('Time')
    ax.set_ylabel('Number of Edges')
    ax.set_title(cond['label'])
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Emergent Property Labels

View the ground truth labels computed from the assembly process.

In [ ]:
print("Emergent Property Labels:")
print("=" * 40)
for key, value in trajectory.labels.items():
    if hasattr(value, 'name'):
        print(f"{key}: {value.name}")
    else:
        print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

## 7. Generate Dataset with Diverse Properties

In [ ]:
# Generate a small dataset
trajectories = generate_dataset(num_samples=50, seed=42)

# Count property classes
from collections import Counter
mechanical_counts = Counter(t.labels['mechanical_class'].name for t in trajectories)

print("Mechanical Property Distribution:")
for label, count in mechanical_counts.items():
    print(f"  {label}: {count}")